# Course 4: Planning
## Part 1: Behaviour Planning in C++
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives
* TODO.

## 1. Introduction

TODO.

## 2. Programming Task

### 2.1. Implement a Cost Function in C++

In previous quizzes, you designed a cost function to choose a lane when trying to reach a goal in the highway driving scenario:
$$
\begin{align}
\mathrm{cost} = 1 - \mathrm{e}^{-\frac{\vert \Delta d \vert}{\Delta s}}.
\end{align}
$$

Here, $\Delta d$ was the lateral distance between the goal lane and the final chosen lane, and $\Delta s$ was the longitudinal distance from the vehicle current position to the goal position. 

In this exercise, we will implement a cost function in C++ that also includes states for planning a lane change right or left (denoted $\mathrm{PLCR}$ and $\mathrm{PLCL}$, respectively). The cost function should incoporate these states. As input to the cost function, we have the following:
* **Intended lane**: the intended lane for the given behaviour. For the $\mathrm{PLCR}$, $\mathrm{PCLC}$, $\mathrm{LCR}$, and $\mathrm{LCL}$, this corresponds to the lane immediately adjacent to the current (one lane over);
* **Final lane**: the immediate resulting lane of the given behaviour. For $\mathrm{LCR}$ and $\mathrm{LCL}$, this would correspond to one lane over;
* **Distance to the goal**: $\Delta s$, the longitudinal distance from the vehicle to the goal;
* **Goal lane**: lane in which the goal marker is situated.

The goal of this exercise is to modify the $\vert \Delta d \vert$ in the equation above so that it satisfies:
* $\vert \Delta d \vert$ decreases when both the intended and final lane are closer to the goal lane;
* Each possible behaviour state $\mathrm{KL}$, $\mathrm{PLCR}$ / $\mathrm{PLCL}$, $\mathrm{LCR}$ / $\mathrm{LCL}$ has a unique cost;
* The values produced by the cost function are in range $[0, 1]$.  

In [ ]:
// From J. Moran's `1_cost.cpp`

In [ ]:
#include <cmath>


double goal_distance_cost(
    int goal_lane, 
    int intended_lane, 
    int final_lane, 
    double distance_to_goal
) {
  // The cost increases with both the distance of intended lane from the goal
  //   and the distance of the final lane from the goal. The cost of being out 
  //   of the goal lane also becomes larger as the vehicle approaches the goal.
    
  /**
   * TODO: Replace cost = 0 with an appropriate cost function.
   */
  double cost = 0;
    
  return cost;
}

In [ ]:
// From J. Moran's `1_tests.cpp`

In [ ]:
void test_goal_distance_cost() {
}